# Testing the trainer code, against extra large model sizes

Going beyond what is possible on a single GPU, 30B and beyond

These requries 8x40GB GPUs with deepspeed 3

In [1]:
# First lets setup the various directories required
!mkdir -p ../../model/
!mkdir -p ../../datapath/
!mkdir -p ../../checkpoint/

In [5]:
# Confgiure the deepspeed / gpu count to be tested
DEEPSPEED_STRAT="deepspeed_stage_3"
GPU_DEVICES="auto"

print("GPU_DEVICES:", GPU_DEVICES)
print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

GPU_DEVICES: auto
DEEPSPEED_STRAT: deepspeed_stage_3
NOTEBOOK_DIR: /home/picocreator/rwkv-proj/infctx-dev/notebook/trainer-validation
TRAINER_DIR: /home/picocreator/rwkv-proj/infctx-dev/RWKV-v4neo
PROJECT_DIR: /home/picocreator/rwkv-proj/infctx-dev


In [6]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/minimal-enwiki.yaml"

Map (num_proc=16): 100%|███████████| 8110/8110 [00:01<00:00, 5340.63 examples/s]
Saving the dataset (1/1 shards): 100%|█| 5318/5318 [00:00<00:00, 127176.21 examp
Saving the dataset (1/1 shards): 100%|█| 54/54 [00:00<00:00, 25572.14 examples/s


## 30B? Size (L80-D5120)

In [ ]:
!cd "{TRAINER_DIR}" && \
    python3 init_model.py \
        --n_layer 80 --n_embd 5120 \
        --vocab_size neox \
        ../model/L80-D5120-neox-init.pth

In [ ]:
!cd "{TRAINER_DIR}" && \
    ls ../model/L80-D5120-neox-init.pth

In [ ]:
!cd "{TRAINER_DIR}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/minimal-enwiki.yaml" \
        --model.load_model "../model/L80-D5120-neox-init.pth" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"